# <center>Algorithmes pour le traitement de séquences.<br/>Alignement optimal et logiciel d’aide à la dététection de plagiat</center>
<center>https://perso.esiee.fr/~coustyj/Sequences/IT-4301E-TDm2.html</center>

<span style="color:red">Importons les packets utiles pour la suite et chargons le contenu des fichier en variable</span>

In [1]:
import numpy as np
t1 = open('t1.txt').read()
t2 = open('t2.txt').read()

## Question 1
Proposer un algorithme en O(|x| × |y|) pour calculer le score d’un alignement optimal entre x et y (indications : on essaiera de se ramener à un problème connu).

<span style="color:red;">Afin de calculer le score d'alignement optimal entre deux string, ici t1 et t2, nous allons utilisé la distance de Levenshtein</span>

## Question 2

On considère la matrice T de taille (|x|+1) × (|y|+1) telle que T[i][j] est le score d’un alignement optimal entre xi et yj, où xi et yj désignent les préfixes de x et de y de longueur i et j respectivement.

Étant donnée la matrice T (obtenue par exemple avec l’algorithme de la question <span style="color:red">1</span>) et les séquences x et y, proposez un schéma de programme retournant un alignement optimal (x′,y′) de x et y.

Évaluer la complexité de calcul de l’algorithme proposé; si celle-ci n’est pas linéaire, proposez un autre algorithme de complexité linéaire.

In [48]:
def levenshtein_distance(a :str, b:str) -> int:
    n,m = len(a),len(b)
    T = np.zeros((n+1,m+1))
    T[:,0] = range(n+1)
    T[0,:] = range(m+1)
    
    for i in range(n):
        for j in range(m):
            same = int(a[i]!=b[j])
            T[i+1,j+1] = min(
                        T[i, j] + same, # NOTE: Substitution
                        T[i, j+1] + 1, # NOTE: Insertion
                        T[i+1, j] + 1  # NOTE: Delete
                    )
            
    return T

# Question 3

Implémentez les algorithmes des questions 1 et 2 et testez les en affichant un alignement optimal de texte1.c et texte2.c

In [50]:
levenshtein_distance(t1,t2)[-1,-1]

780.0

In [52]:
levenshtein_distance(open('texte1.txt').read(),open('texte2.txt').read())[:,-1]

array([1801., 1800., 1799., ...,  579.,  578.,  577.])

In [49]:
levenshtein_distance('a','b')

array([[0., 1.],
       [1., 1.]])